In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt

from hda_fits.pink import write_mosaic_objects_to_pink_file_v2
import hda_fits.fits as hf


plt.rcParams["figure.figsize"] = (12, 8)

## Create table subset including P205+55 entries

In [3]:
tab = hf.read_shimwell_catalog("../data/LOFAR_HBA_T1_DR1_catalog_v1.0.srl.fits", reduced=True)
tab = tab[tab.Mosaic_ID.str.contains("P205")].loc[:, ["Source_Name", "RA", "DEC", "Mosaic_ID"]]

In [4]:
tab

,Source_Name,RA,DEC,Mosaic_ID
182259,ILTJ134917.04+535737.2,207.320991,53.960334,P205+55
182260,ILTJ134916.45+535801.7,207.318522,53.967151,P205+55
182261,ILTJ134915.23+540433.5,207.313443,54.075982,P205+55
182262,ILTJ134915.03+540812.1,207.312622,54.136707,P205+55
182263,ILTJ134911.34+540239.4,207.297256,54.044286,P205+55
...,...,...,...,...
189028,ILTJ134000.89+562526.5,205.003718,56.424033,P205+55
189029,ILTJ133759.61+550106.4,204.498375,55.018459,P205+55
189030,ILTJ133614.95+544711.3,204.062292,54.786475,P205+55
189031,ILTJ133340.22+540139.8,203.417603,54.027725,P205+55


In [5]:
hdu = hf.load_mosaic("P205+55", "../data/")[0]

2021-10-30 18:41:11,092 fits.py: Loading ../data/P205+55-mosaic.fits


In [6]:
tab = tab.reset_index(drop=True)
coords = tab.loc[:1000, ["RA", "DEC"]].values.tolist()
coords2 = tab.loc[1001:2000, ["RA", "DEC"]].values.tolist()

In [7]:
write_mosaic_objects_to_pink_file_v2(
    filepath="../data/P205+55_pink_v2.bin",
    hdu=hdu,
    coordinates=coords,
    image_size=200
)

2021-10-30 18:41:21,514 pink.py: Wrote 1001 images to ../data/P205+55_pink_v2.bin


'../data/P205+55_pink_v2.bin'

In [8]:
write_mosaic_objects_to_pink_file_v2(
    filepath="../data/P205+55_pink_v2_2.bin",
    hdu=hdu,
    coordinates=coords2,
    image_size=200
)

2021-10-30 18:41:25,695 pink.py: Wrote 1000 images to ../data/P205+55_pink_v2_2.bin


'../data/P205+55_pink_v2_2.bin'